In [13]:
import numpy as np
import os
from sklearn.metrics import confusion_matrix
import seaborn as sn; sn.set(font_scale=1.4)
from sklearn.utils import shuffle           
import matplotlib.pyplot as plt             
import cv2                                 
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation, BatchNormalization, Dropout, MaxPool2D, GlobalAveragePooling2D
from tqdm import tqdm
import time
import random


In [2]:
data = np.load('data_bw.npz')
trainX,  trainy, testX, testy = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']
print('Loaded: ', trainX.shape, trainy.shape, testX.shape, testy.shape)

Loaded:  (1065, 160, 160, 3) (1065,) (452, 160, 160, 3) (452,)


In [7]:
class_names = ['Arjun_Rampal', 'Arshad_Warsi', 'Asin', 'Ayushmann_Khurrana', 'Bhumi_Pednekar', 'Bipasha_Basu', 'Bobby_Deol', 'Deepika_Padukone', 'Disha_Patani', 'Emraan_Hashmi', 'Esha_Gupta', 'Farhan_Akhtar', 'Govinda']
class_names_label = {class_name:i for i, class_name in enumerate(class_names)}
nb_classes = len(class_names)
IMAGE_SIZE = (160, 160)

In [8]:
n_train = trainy.shape[0]
n_test = testy.shape[0]

print ("Number of training examples: {}".format(n_train))
print ("Number of testing examples: {}".format(n_test))
print ("Each face is of size: {}".format(IMAGE_SIZE))

Number of training examples: 1065
Number of testing examples: 452
Each face is of size: (160, 160)


In [3]:
trainX = trainX / 255
testX = testX / 255

In [5]:
random.seed(4012)
trainX, trainy = shuffle(trainX, trainy, random_state=4012)

In [6]:
trainy

array(['Disha_Patani', 'Bhumi_Pednekar', 'Govinda', ..., 'Bhumi_Pednekar',
       'Emraan_Hashmi', 'Ayushmann_Khurrana'], dtype='<U18')

In [9]:
# one-hot encoding
# perform one-hot encoding on train label
from sklearn.preprocessing import OneHotEncoder 
print(trainy.shape) #a list without second shape
print(trainy.reshape(-1, 1).shape) #change to a 2d array
encoder = OneHotEncoder()
trainy_hot = encoder.fit_transform(trainy.reshape(-1,1))
trainy_hot 
# trainy_hot is a sparse matrix, convert it to an ndarray
trainy_hot.toarray()

(1065,)
(1065, 1)


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [10]:
# perform one hot encoding on test label
from sklearn.preprocessing import OneHotEncoder 
encoder = OneHotEncoder()
test_labels_hot = encoder.fit_transform(testy.reshape(-1,1))
test_labels_hot 
test_labels_hot.toarray()

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]])

#### **Try a model offered by Kaggle face recognition**

In [11]:
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

tf.random.set_seed(4012)


model = Sequential()

model.add(Conv2D(filters = 20, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (160,160,3)))

model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 50, kernel_size = (6,6),padding = 'Same', 
                 activation ='relu'))

model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 150, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (160,160,3)))

model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(13, activation = "softmax"))

optimizer = 'adam'
model.compile(optimizer = optimizer , loss = "binary_crossentropy", metrics=["accuracy"])

learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', 
                      patience=3, 
                      verbose=1, 
                      factor=0.7, 
                      min_lr=0.00000000001)

epoch = 50
batch_size = 32

datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=5,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.05, # Randomly zoom image 
        width_shift_range=0,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images
datagen.fit(trainX)

# 直接指明test set作为validation，test accuracy最好可以达到65%+
history = model.fit(
                datagen.flow(trainX,trainy_hot.toarray(), batch_size=batch_size),
                epochs = epoch, 
                validation_data = (testX, test_labels_hot.toarray()),
                verbose = 2, 
                steps_per_epoch=trainX.shape[0] // batch_size,
                callbacks=[learning_rate_reduction]
                )

Epoch 1/50
33/33 - 12s - loss: 0.4656 - accuracy: 0.0862 - val_loss: 0.2759 - val_accuracy: 0.1283 - lr: 0.0010 - 12s/epoch - 363ms/step
Epoch 2/50
33/33 - 5s - loss: 0.2948 - accuracy: 0.1042 - val_loss: 0.2908 - val_accuracy: 0.1283 - lr: 0.0010 - 5s/epoch - 152ms/step
Epoch 3/50
33/33 - 5s - loss: 0.2873 - accuracy: 0.0997 - val_loss: 0.2675 - val_accuracy: 0.1283 - lr: 0.0010 - 5s/epoch - 147ms/step
Epoch 4/50
33/33 - 5s - loss: 0.2641 - accuracy: 0.1704 - val_loss: 0.2386 - val_accuracy: 0.2478 - lr: 0.0010 - 5s/epoch - 146ms/step
Epoch 5/50
33/33 - 5s - loss: 0.2471 - accuracy: 0.2488 - val_loss: 0.2324 - val_accuracy: 0.2898 - lr: 0.0010 - 5s/epoch - 144ms/step
Epoch 6/50
33/33 - 4s - loss: 0.2286 - accuracy: 0.3311 - val_loss: 0.2093 - val_accuracy: 0.3827 - lr: 0.0010 - 4s/epoch - 121ms/step
Epoch 7/50
33/33 - 4s - loss: 0.2145 - accuracy: 0.3863 - val_loss: 0.2004 - val_accuracy: 0.4292 - lr: 0.0010 - 4s/epoch - 116ms/step
Epoch 8/50
33/33 - 4s - loss: 0.2003 - accuracy: 0.44

### **Keras预训练模型：https://keras.io/api/applications/**

#### **mobilenet**

In [ ]:
# 定义一个生成器，在读取数据的时候提前将图像高和宽扩大到模型要求的图像高和宽，符合网络的输入需求

from skimage.transform import resize
# dataGenerator后面会反复用到
# batch size是批量的数据大小，函数的作用是把数据本身的size变成resize_size的大小
def dataGenerator(X_train, y_train, batch_size, resize_size=(160, 160, 3)):    
    total_size = X_train.shape[0]
    
    while True:
        permutation = list(np.random.permutation(total_size))
        for i in range(total_size // batch_size):
            index = permutation[i * batch_size : (i + 1) * batch_size]
            X_batch = X_train[index]
            y_batch = y_train[index]
            
            if resize_size is not None:
                X_batch = resize(X_batch, (X_batch.shape[0], *resize_size))
                    
            yield X_batch, y_batch


In [ ]:
from tensorflow.keras.applications import MobileNet

# 先设置一个mobilenet的base model
# input shape与mtcnn输出设成一致
base_model = MobileNet(input_shape=(160, 160, 3),  # MobileNet要求输入的尺寸不小于32，不同模型有不同要求
                include_top=False,  # 不使用最后一层，因为我们要按照具体的数据来设置最后一层的神经元个数。原来的最后一层是对应一个1000类的分类问题
                weights="imagenet")  # 使用预训练的权重可以提高准确率

# 添加a global spatial average pooling layer
x = base_model.output
# 取每一个通道的平均值作为输出
x = GlobalAveragePooling2D()(x)
# 添加a fully-connected layer
x = Dense(1024, activation='relu')(x)
# 更改为我们的13类分类问题
predictions = Dense(13, activation='softmax')(x)

# 这里没有用到flatten的操作，被替代了。flatten相当于输出串联拼接，然后再通过全连接层输出，而global average pooling是每一层直接取average得到对应的结果
                       
# 我们最终会训练的模型
net = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
base_model.input

<KerasTensor: shape=(None, 160, 160, 3) dtype=float32 (created by layer 'input_3')>

In [ ]:
# 训练的过程跟上面类似，注意输入图像的尺寸需要进行转换
batch_size = 32
resize_size = (160, 160, 3)
datagen = dataGenerator(trainX, trainy_hot.toarray(), batch_size, resize_size)

optimizer = 'adam'

net.compile(optimizer=optimizer,
            loss='binary_crossentropy',
            metrics=['accuracy'])

epochs = 50

tf.random.set_seed(4012)

net.fit(datagen,
        steps_per_epoch = len(trainX) // batch_size,
        epochs=epochs,
        verbose=1)

Epoch 1/50
33/33 [==============================] - 108s 3s/step - loss: 0.2237 - accuracy: 0.4972
Epoch 2/50
33/33 [==============================] - 103s 3s/step - loss: 0.0386 - accuracy: 0.9347
Epoch 3/50
33/33 [==============================] - 103s 3s/step - loss: 0.0135 - accuracy: 0.9830
Epoch 4/50
33/33 [==============================] - 103s 3s/step - loss: 0.0081 - accuracy: 0.9877
Epoch 5/50
33/33 [==============================] - 103s 3s/step - loss: 0.0089 - accuracy: 0.9886
Epoch 6/50
33/33 [==============================] - 103s 3s/step - loss: 0.0116 - accuracy: 0.9830
Epoch 7/50
33/33 [==============================] - 103s 3s/step - loss: 0.0085 - accuracy: 0.9867
Epoch 8/50
33/33 [==============================] - 103s 3s/step - loss: 0.0097 - accuracy: 0.9848
Epoch 9/50
33/33 [==============================] - 103s 3s/step - loss: 0.0224 - accuracy: 0.9631
Epoch 10/50
33/33 [==============================] - 104s 3s/step - loss: 0.0230 - accuracy: 0.9593
Epoch 11/

KeyboardInterrupt: ignored

In [ ]:
# 同样转换测试集的尺寸
batch_size = 32
resize_size = (160, 160, 3)
test_datagen = dataGenerator(testX, test_labels_hot.toarray(), batch_size, resize_size)

# 仅fit十轮终止，test acc达到66.5%
loss, acc = net.evaluate(test_datagen, steps=len(testX) // batch_size)
acc

14/14 [==============================] - 14s 948ms/step - loss: 0.2863 - accuracy: 0.6652


0.6651785969734192

#### **Xception**

In [ ]:
# 完全同理
from tensorflow.keras.applications import Xception

base_model2 = Xception(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=(160, 160, 3),
    pooling=None,
    classifier_activation="softmax",
)
x = base_model2.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(13, activation='softmax')(x)
net = Model(inputs=base_model2.input, outputs=predictions)

In [ ]:
batch_size = 32
resize_size = (160, 160, 3)
datagen = dataGenerator(trainX, trainy_hot.toarray(), batch_size, resize_size)
optimizer = 'adam'

net.compile(optimizer=optimizer,
            loss='binary_crossentropy',
            metrics=['accuracy'])
            
epochs = 15

tf.random.set_seed(4012)

net.fit(datagen,
        steps_per_epoch = len(trainX) // batch_size,
        epochs=epochs,
        verbose=1)

Epoch 1/15
33/33 [==============================] - 438s 13s/step - loss: 0.2704 - accuracy: 0.2225
Epoch 2/15
33/33 [==============================] - 428s 13s/step - loss: 0.1163 - accuracy: 0.6856
Epoch 3/15
33/33 [==============================] - 429s 13s/step - loss: 0.0538 - accuracy: 0.8826
Epoch 4/15
33/33 [==============================] - 431s 13s/step - loss: 0.0411 - accuracy: 0.9034
Epoch 5/15
33/33 [==============================] - 434s 13s/step - loss: 0.0419 - accuracy: 0.9081
Epoch 6/15
33/33 [==============================] - 434s 13s/step - loss: 0.0270 - accuracy: 0.9479
Epoch 7/15
33/33 [==============================] - 439s 13s/step - loss: 0.0235 - accuracy: 0.9527
Epoch 8/15
33/33 [==============================] - 443s 13s/step - loss: 0.0146 - accuracy: 0.9754
Epoch 9/15
33/33 [==============================] - 428s 13s/step - loss: 0.0145 - accuracy: 0.9678
Epoch 10/15
33/33 [==============================] - 427s 13s/step - loss: 0.0113 - accuracy: 0.9763

In [ ]:
# 训练15轮的xception的效果达到83.5%
batch_size = 32
resize_size = (160, 160, 3)
test_datagen = dataGenerator(testX, test_labels_hot.toarray(), batch_size, resize_size)

loss, acc = net.evaluate(test_datagen, steps=len(testX) // batch_size)
acc

14/14 [==============================] - 46s 3s/step - loss: 0.0927 - accuracy: 0.8348


0.8348214030265808

#### **ResNet**

In [ ]:
from tensorflow.keras.applications import ResNet101V2

base_model3 = ResNet101V2(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=(160, 160, 3),
    pooling=None,
    classifier_activation="softmax",
)
x = base_model3.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(13, activation='softmax')(x)
                       
net = Model(inputs=base_model3.input, outputs=predictions)

171327488/171317808 [==============================] - 3s 0us/step


In [ ]:
batch_size = 32
resize_size = (160, 160, 3)
datagen = dataGenerator(trainX, trainy_hot.toarray(), batch_size, resize_size)

optimizer = 'adam'

net.compile(optimizer=optimizer,
            loss='binary_crossentropy',
            metrics=['accuracy'])

epochs = 15

net.fit(datagen,
        steps_per_epoch = len(trainX) // batch_size,
        epochs=epochs,
        verbose=1)


Epoch 1/15
33/33 [==============================] - 640s 19s/step - loss: 0.2407 - accuracy: 0.3305
Epoch 2/15
33/33 [==============================] - 611s 19s/step - loss: 0.1586 - accuracy: 0.5559
Epoch 3/15
33/33 [==============================] - 608s 18s/step - loss: 0.1244 - accuracy: 0.6714
Epoch 4/15
33/33 [==============================] - 612s 19s/step - loss: 0.0923 - accuracy: 0.7850
Epoch 5/15
33/33 [==============================] - 614s 19s/step - loss: 0.0761 - accuracy: 0.8191
Epoch 6/15
33/33 [==============================] - 612s 19s/step - loss: 0.0712 - accuracy: 0.8362
Epoch 7/15
33/33 [==============================] - 615s 19s/step - loss: 0.0512 - accuracy: 0.8892
Epoch 8/15
33/33 [==============================] - 611s 19s/step - loss: 0.0290 - accuracy: 0.9328
Epoch 9/15
33/33 [==============================] - 612s 19s/step - loss: 0.0404 - accuracy: 0.9176
Epoch 10/15
33/33 [==============================] - 606s 18s/step - loss: 0.0180 - accuracy: 0.9593

In [ ]:
# resnet效果不行
batch_size = 32
resize_size = (160, 160, 3)
test_datagen = dataGenerator(testX, test_labels_hot.toarray(), batch_size, resize_size)

loss, acc = net.evaluate(test_datagen, steps=len(testX) // batch_size)
acc

14/14 [==============================] - 64s 4s/step - loss: 0.4941 - accuracy: 0.4308


0.4308035671710968